In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
sns.set()

import torch.nn.functional as F
from torch.autograd import Variable
import torch

from torch import nn

In [2]:
df = pd.read_csv('Data/AAPL.csv')
date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-05-19,85.407143,86.761429,85.332855,86.370003,79.160240,79438800
1,2014-05-20,86.358574,86.628571,85.818573,86.387146,79.175926,58709000
2,2014-05-21,86.261429,86.671425,86.008568,86.615715,79.385437,49214900
3,2014-05-22,86.657143,87.121429,86.300003,86.752853,79.511116,50190000
4,2014-05-23,86.750000,87.818573,86.638573,87.732857,80.409317,58052400


In [25]:
minmax = MinMaxScaler().fit(df.iloc[:, 1:].astype('float32'))
df_log = minmax.transform(df.iloc[:, 1:].astype('float32'))
df_log = pd.DataFrame(df_log)
df_log.head()

/Users/polina/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,0,1,2,3,4,5
0,0.000000,0.000905,0.000000,0.000000,0.000000,0.381021
1,0.006545,0.000000,0.003363,0.000118,0.000104,0.264803
2,0.005877,0.000292,0.004678,0.001686,0.001499,0.211577
3,0.008599,0.003356,0.006696,0.002628,0.002336,0.217043
4,0.009237,0.008104,0.009039,0.009354,0.008314,0.261122


In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs


# if __name__ == '__main__':
#     # set random seed to 0
#     np.random.seed(0)
#     torch.manual_seed(0)
#     # load data and make training set
#     data = torch.load('traindata.pt')
#     input = torch.from_numpy(data[3:, :-1])
#     target = torch.from_numpy(data[3:, 1:])
#     test_input = torch.from_numpy(data[:3, :-1])
#     test_target = torch.from_numpy(data[:3, 1:])
#     # build the model
#     seq = Sequence()
#     seq.double()
#     criterion = nn.MSELoss()
#     # use LBFGS as optimizer since we can load the whole data to train
#     optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
#     #begin to train
#     for i in range(15):
#         print('STEP: ', i)
#         def closure():
#             optimizer.zero_grad()
#             out = seq(input)
#             loss = criterion(out, target)
#             print('loss:', loss.item())
#             loss.backward()
#             return loss
#         optimizer.step(closure)
#         # begin to predict, no need to track gradient here
#         with torch.no_grad():
#             future = 1000
#             pred = seq(test_input, future=future)
#             loss = criterion(pred[:, :-future], test_target)
#             print('test loss:', loss.item())
#             y = pred.detach().numpy()
#         # draw the result
#         plt.figure(figsize=(30,10))
#         plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
#         plt.xlabel('x', fontsize=20)
#         plt.ylabel('y', fontsize=20)
#         plt.xticks(fontsize=20)
#         plt.yticks(fontsize=20)
#         def draw(yi, color):
#             plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
#             plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
#         draw(y[0], 'r')
#         draw(y[1], 'g')
#         draw(y[2], 'b')
#         plt.savefig('predict%d.pdf'%i)
#         plt.close()

/Users/polina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/polina/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/polina/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/polina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/polina/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/polina/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 497, in s

FileNotFoundError: [Errno 2] No such file or directory: 'traindata.pt'

In [81]:
def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    layernorm = nn.LayerNorm(x.size(), eps=1e-8)
    x = layernorm(x)
    pad = torch.zeros([x.size()[0], pad_sz, hidden_dim])
    print(x.size(), pad.size())
    x = torch.cat([pad, x, pad], 1)
    
    layer_conv = torch.nn.Conv1d(in_channels = x.size()[1],
                        out_channels = hidden_dim,
                        kernel_size = kernel_size,
                        dilation = dilation_rate)
    x = layer_conv(x)
    x = x[:, :-pad_sz, :]
    x = F.relu(x)
    return x

In [82]:
batch_x = np.expand_dims(df_log.iloc[0 : 100, :].values, axis = 0)

In [83]:
batch_x.shape

(1, 100, 6)

In [84]:
x = Variable(torch.tensor(batch_x, dtype=torch.float32))

In [85]:
size_layer = 6

In [86]:
x.shape[1]

100

In [87]:
for i in range(num_layers):
    dilation_rate = 2 ** i
    pad_sz = (kernel_size - 1) * dilation_rate
    x += cnn_block(x, dilation_rate, pad_sz, size_layer, kernel_size)
print(x)

torch.Size([1, 100, 6]) torch.Size([1, 1, 6])


RuntimeError: The size of tensor a (6) must match the size of tensor b (5) at non-singleton dimension 2

In [103]:
class ByteNet(nn.Sequential):

    def __init__(self,
             learning_rate = 1e-4,
             num_layers = 1,
             size = df_log.shape[1],
             size_layer = 128,
             output_size = df_log.shape[1],
             kernel_size = 2,
             dropout = 0.9):
        
        super(ByteNet, self).__init__()
        
        self.add_module('conv1', nn.Conv1d(size, size_layer, 1))
        self.add_module('do1', nn.Dropout(dropout))
        
#         for s in range(num_sets):
#             for r in dilation_rates:
#                 self.add_module('block%s_%s' % (s, r),
#                                 block(num_channels, input_channel,kernel_size=kernel_size,
#                                       dilation=r, causal=causal))

In [ ]:
class Model(nn.Module):
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        kernel_size = 2,
        dropout = 0.9):
        
        self.learning_rate = learning_rate
        self.num_layers = num_layers
        self.size = size
        self.size_layer = size_layer
        self.output_size = output_size
        self.kernel_size = kernel_size
        self.dropout = dropout
        
        
        self.conv1 = nn.Conv1d()
        
        x = tf.layers.conv1d(self.X, size_layer, 1)
        x = tf.nn.dropout(x, keep_prob = dropout)
        
        
        
        
        for i in range(num_layers):
            dilation_rate = 2 ** i
            pad_sz = (kernel_size - 1) * dilation_rate
            with tf.variable_scope('block_%d'%i):
                x += cnn_block(x, dilation_rate, pad_sz, size_layer, kernel_size)
        print(x)
        
        def lstm_cell(size_layer):
            return tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False)

#         torch.nn.RNNCell(input_size, hidden_size, bias=True, nonlinearity='tanh')
        
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell(
            [lstm_cell(size_layer) for _ in range(num_layers)],
            state_is_tuple = False,
        )
        drop = tf.contrib.rnn.DropoutWrapper(
            rnn_cells, output_keep_prob = dropout
        )
        self.hidden_layer = tf.placeholder(
            tf.float32, (None, num_layers * 2 * size_layer)
        )
        
        self.outputs, self.last_state = tf.nn.dynamic_rnn(
            drop, x, initial_state = self.hidden_layer, dtype = tf.float32
        )
        self.logits = tf.layers.dense(self.outputs[-1], output_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        optimizer = tf.train.AdamOptimizer(learning_rate)
        gradients, variables = zip(*optimizer.compute_gradients(self.cost))
        gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
        self.optimizer = optimizer.apply_gradients(zip(gradients, variables))

In [104]:
model = ByteNet()
num_layers = 1
size_layer = 128
timestamp = 30
epoch = 10
future_day = 50

cost = nn.MSELoss()
opt = torch.optim.Adam(model.parameters(), 0.001)

In [105]:
for i in range(epoch):
    total_loss = 0
    #losses=[]
    for k in range(0, df_log.shape[0]-1 , timestamp):
        
        index = min(k + timestamp, df_log.shape[0] -1)
        batch_x = np.expand_dims(
            df_log.iloc[index - timestamp : index, :].values, axis = 0
        )
        #print('batch',batch_x.shape)
        batch_y = df_log.iloc[index - timestamp + 1 : index + 1, :].values
#         print(batch_x[0].shape)
        X = Variable(torch.tensor(batch_x, dtype = torch.float32))
        Y = Variable(torch.tensor(batch_y, dtype = torch.float32))
       
        prediction = model.forward(X)
#         print(prediction.size())
        loss = cost(Y, prediction)

        loss.backward()
        opt.step()
        opt.zero_grad()

        total_loss += loss
    total_loss /= df_log.shape[0] // timestamp
    losses.append(total_loss.detach().detach().numpy())
    if (i + 1) % 100 == 0:
        print('epoch:', i + 1, 'avg loss:', total_loss.detach().numpy())

RuntimeError: Given groups=1, weight of size [128, 6, 1], expected input[1, 30, 6] to have 6 channels, but got 30 channels instead

In [14]:
layer = torch.nn.Conv1d(x.shape[1], size_layer, 1)
x = layer(x)

In [15]:
x

tensor([[[ 0.0016, -0.0005,  0.0011, -0.0049, -0.0052, -0.0348],
         [-0.1271, -0.1237, -0.1287, -0.1276, -0.1232, -0.1530],
         [-0.0327, -0.0315, -0.0305, -0.0264, -0.0243, -0.0960],
         [-0.0007, -0.0014, -0.0043, -0.0034, -0.0113,  0.2168],
         [-0.0416, -0.0381, -0.0369, -0.0347, -0.0337, -0.0378],
         [ 0.0658,  0.0691,  0.0661,  0.0679,  0.0677,  0.1478],
         [-0.0319, -0.0329, -0.0360, -0.0362, -0.0384, -0.0265],
         [-0.0939, -0.0883, -0.0922, -0.0882, -0.0868, -0.0778],
         [ 0.0200,  0.0161,  0.0184,  0.0181,  0.0143,  0.0756],
         [-0.0494, -0.0489, -0.0442, -0.0428, -0.0409, -0.0400]]],
       grad_fn=<SqueezeBackward1>)

In [16]:
dropout = torch.nn.Dropout(0.5)
x = dropout(x)

In [17]:
x

tensor([[[ 0.0000, -0.0011,  0.0000, -0.0098, -0.0000, -0.0697],
         [-0.0000, -0.0000, -0.0000, -0.0000, -0.2463, -0.0000],
         [-0.0000, -0.0000, -0.0000, -0.0528, -0.0000, -0.0000],
         [-0.0000, -0.0028, -0.0085, -0.0000, -0.0226,  0.4335],
         [-0.0832, -0.0000, -0.0000, -0.0693, -0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.1359,  0.1355,  0.0000],
         [-0.0638, -0.0000, -0.0000, -0.0000, -0.0000, -0.0530],
         [-0.1878, -0.1765, -0.0000, -0.0000, -0.1736, -0.1556],
         [ 0.0000,  0.0322,  0.0368,  0.0362,  0.0286,  0.0000],
         [-0.0987, -0.0977, -0.0885, -0.0857, -0.0818, -0.0000]]],
       grad_fn=<MulBackward0>)

In [18]:
num_layers = 5
kernel_size = 2

In [19]:
def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    layernorm = nn.LayerNorm(x.size(), eps=1e-8)
    x = layernorm(x)
#     pad = torch.zeros([pad_sz, hidden_dim, x.shape[0]])
#     x = torch.cat([pad, x, pad], 1)
    layer_conv = torch.nn.Conv1d(in_channels = x.shape[0],
                        out_channels = hidden_dim,
                        kernel_size = kernel_size,
                        padding = pad_sz,
                        dilation = dilation_rate)
    x = layer_conv(x)
    print(x.shape)
    x = x[:, :-pad_sz, :]
    x = torch.nn.ReLU(x)
    return x

In [20]:
layernorm = nn.LayerNorm(x.size(), eps=1e-8)
x = layernorm(x)

In [21]:
x.shape

torch.Size([1, 10, 6])

In [22]:
pad = torch.zeros([pad_sz, size_layer, x.shape[0]])

NameError: name 'pad_sz' is not defined

In [87]:
pad.shape

torch.Size([1, 10, 1])

In [80]:
pad

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])

In [79]:
x = torch.cat([pad, x, pad], 1)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 10 and 6 in dimension 2 at /Users/soumith/b101_2/2019_02_08/wheel_build_dirs/wheel_3.7/pytorch/aten/src/TH/generic/THTensorMoreMath.cpp:1307

In [95]:
for i in range(num_layers):
    dilation_rate = 2 ** i
    pad_sz = (kernel_size - 1) * dilation_rate
    x += cnn_block(x, dilation_rate, pad_sz, size_layer, kernel_size)
print(x)

RuntimeError: Given groups=1, weight of size [10, 1, 2], expected input[1, 10, 6] to have 1 channels, but got 10 channels instead

In [ ]:
cnn_block(x, dilation_rate, pad_sz, size_layer, kernel_size)